In [1]:
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K
from keras.layers import * 
from keras import layers, initializers, optimizers
from keras import models
from datetime import datetime

from Dental_Tool.Data_processing import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v2 import *

import keras.backend as K 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
import time

Using TensorFlow backend.


In [2]:
directory = [ 
                "Dental_Data/PBL/10_20200901", "Dental_Data/PBL/10_20200901_Flip", 
                "Dental_Data/PBL/10_clahe_20200901", "Dental_Data/PBL/10_clahe_20200901_Flip"
            ]


directory = [ i + "/mapping.json" for i in directory]
argscale_num = 80

data = load_json(directory)
dataset = json_2_dataframe_PBL(data)
dataset

,Path,State,Class,source,tooth_num,angle
0,Dental_Data/PBL/10_20200901/15-53-18-500_00040...,1,0,NN_191024_151623_BE78A8_6,6,-10
1,Dental_Data/PBL/10_20200901_Flip/15-53-18-500_...,1,0,NN_191024_151623_BE78A8_6,6,-10
2,Dental_Data/PBL/10_clahe_20200901/16-31-08-628...,1,0,NN_191024_151623_BE78A8_6,6,-10
3,Dental_Data/PBL/10_clahe_20200901_Flip/16-31-0...,1,0,NN_191024_151623_BE78A8_6,6,-10
4,Dental_Data/PBL/10_20200901/15-53-18-515_00040...,1,0,NN_191024_151623_BE78A8_6,6,-9
...,...,...,...,...,...,...
381435,Dental_Data/PBL/10_clahe_20200901_Flip/17-03-2...,2,1,NN_180917_113933_C0A0B2_26,26,8
381436,Dental_Data/PBL/10_20200901/16-30-51-036_S5949...,2,1,NN_180917_113933_C0A0B2_26,26,9
381437,Dental_Data/PBL/10_20200901_Flip/16-30-51-036_...,2,1,NN_180917_113933_C0A0B2_26,26,9
381438,Dental_Data/PBL/10_clahe_20200901/17-03-20-150...,2,1,NN_180917_113933_C0A0B2_26,26,9


In [3]:
# import keras.backend as K
# import tensorflow as tf
# from keras import initializers, layers

# class Length(layers.Layer):
#     """
#     Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss
#     inputs: shape=[dim_1, ..., dim_{n-1}, dim_n]
#     output: shape=[dim_1, ..., dim_{n-1}]
#     """
#     def call(self, inputs, **kwargs):
#         return K.sqrt(K.sum(K.square(inputs), -1))

#     def compute_output_shape(self, input_shape):
#         return input_shape[:-1]

# class Mask(layers.Layer):
#     """
#     Mask a Tensor with shape=[None, d1, d2] by the max value in axis=1.
#     Output shape: [None, d2]
#     """
#     def call(self, inputs, **kwargs):
#         # use true label to select target capsule, shape=[batch_size, num_capsule]
#         if type(inputs) is list:  # true label is provided with shape = [batch_size, n_classes], i.e. one-hot code.
#             assert len(inputs) == 2
#             inputs, mask = inputs
#         else:  # if no true label, mask by the max length of vectors of capsules
#             x = inputs
#             # Enlarge the range of values in x to make max(new_x)=1 and others < 0
#             x = (x - K.max(x, 1, True)) / K.epsilon() + 1
#             mask = K.clip(x, 0, 1)  # the max value in x clipped to 1 and other to 0

#         # masked inputs, shape = [batch_size, dim_vector]
#         inputs_masked = K.batch_dot(inputs, mask, [1, 1])
#         return inputs_masked

#     def compute_output_shape(self, input_shape):
#         if type(input_shape[0]) is tuple:  # true label provided
#             return tuple([None, input_shape[0][-1]])
#         else:
#             return tuple([None, input_shape[-1]])


# def squash(vectors, axis=-1):
#     """
#     The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
#     :param vectors: some vectors to be squashed, N-dim tensor
#     :param axis: the axis to squash
#     :return: a Tensor with same shape as input vectors
#     """
#     s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
#     scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm)
#     return scale * vectors


# class CapsuleLayer(layers.Layer):
#     """
#     The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
#     neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
#     from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_vector] and output shape = \
#     [None, num_capsule, dim_vector]. For Dense Layer, input_dim_vector = dim_vector = 1.
    
#     :param num_capsule: number of capsules in this layer
#     :param dim_vector: dimension of the output vectors of the capsules in this layer
#     :param num_routings: number of iterations for the routing algorithm
#     """
#     def __init__(self, num_capsule, dim_vector, num_routing=3,
#                  kernel_initializer='glorot_uniform',
#                  bias_initializer='zeros',
#                  **kwargs):
#         super(CapsuleLayer, self).__init__(**kwargs)
#         self.num_capsule = num_capsule
#         self.dim_vector = dim_vector
#         self.num_routing = num_routing
#         self.kernel_initializer = initializers.get(kernel_initializer)
#         self.bias_initializer = initializers.get(bias_initializer)

#     def build(self, input_shape):
#         assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_vector]"
#         self.input_num_capsule = input_shape[1]
#         self.input_dim_vector = input_shape[2]

#         # Transformation matrix/Weight matrix
#         self.W = self.add_weight(shape=[self.input_num_capsule, self.num_capsule, self.input_dim_vector, self.dim_vector],
#                                  initializer=self.kernel_initializer,
#                                  name='W')

#         # Coupling coefficient. The redundant dimensions are just to facilitate subsequent matrix calculation.
#         self.bias = self.add_weight(shape=[1, self.input_num_capsule, self.num_capsule, 1, 1],
#                                     initializer=self.bias_initializer,
#                                     name='bias',
#                                     trainable=False)
#         self.built = True
        

            
            
            
# #     def call(self, inputs, training=None):
# #         # inputs.shape=[None, input_num_capsule, input_dim_vector]
# #         # Expand dims to [None, input_num_capsule, 1, 1, input_dim_vector]
# #         inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)

# #         # Replicate(tile) num_capsule dimension to prepare being multiplied by W
# #         # Now it has shape = [None, input_num_capsule, num_capsule, 1, input_dim_vector]
# #         inputs_tiled = K.tile(inputs_expand, [1, 1, self.num_capsule, 1, 1])

# #         """  
# #         # Compute `inputs * W` by expanding the first dim of W. More time-consuming and need batch_size.
# #         # Now W has shape  = [batch_size, input_num_capsule, num_capsule, input_dim_vector, dim_vector]
# #         w_tiled = K.tile(K.expand_dims(self.W, 0), [self.batch_size, 1, 1, 1, 1])
        
# #         # Transformed vectors, inputs_hat.shape = [None, input_num_capsule, num_capsule, 1, dim_vector]
# #         inputs_hat = K.batch_dot(inputs_tiled, w_tiled, [4, 3])
# #         """
        
# #         # inputs_hat.shape = [None, input_num_capsule, num_capsule, 1, dim_vector]
# #         inputs_hat = tf.scan(lambda ac, x: K.batch_dot(x, self.W, [3, 2]),
# #                              elems=inputs_tiled,
# #                              initializer=K.zeros([self.input_num_capsule, self.num_capsule, 1, self.dim_vector]))

# #         # DYNAMIC ROUTING
# #         assert self.num_routing > 0, 'The num_routing should be > 0.'
# #         for i in range(self.num_routing):
# #             c = tf.nn.softmax(self.bias, dim=2)  # dim=2 is the num_capsule dimension
# #             outputs = squash(K.sum(c * inputs_hat, 1, keepdims=True))

# #             # last iteration needs not compute bias which will not be passed to the graph any more anyway.
# #             if i != self.num_routing - 1:
# #                 # update the raw weights for the next routing iteration
# #                 # by adding the agreement to the previous raw weights
# #                 self.bias += K.sum(inputs_hat * outputs, -1, keepdims=True)

# #         return K.reshape(outputs, [-1, self.num_capsule, self.dim_vector])

#     def compute_output_shape(self, input_shape):
#             return tuple([None, self.num_capsule, self.dim_vector])


# def PrimaryCap(inputs, dim_vector, n_channels, kernel_size, strides, padding):
#     """
#     Apply Conv2D `n_channels` times and concatenate all capsules
#     :param inputs: 4D tensor, shape=[None, width, height, channels]
#     :param dim_vector: the dim of the output vector of capsule
#     :param n_channels: the number of types of capsules
#     :return: output tensor, shape=[None, num_capsule, dim_vector]
#     """
#     output = layers.Conv2D(filters=dim_vector*n_channels, kernel_size=kernel_size, strides=strides, padding=padding, name='primary_caps_conv')(inputs)
#     outputs = layers.Reshape(target_shape=[-1, dim_vector], name='primary_caps_reshape')(output)
#     return layers.Lambda(squash)(outputs)



# from keras import layers, models
# from keras import backend as K
# from keras.utils import to_categorical
# def CapsNet(input_shape, n_class, num_routing):
#     """
#     A Capsule Network on MNIST.
#     :param input_shape: data shape, 4d, [None, width, height, channels]
#     :param n_class: number of classes
#     :param num_routing: number of routing iterations
#     :return: A Keras Model with 2 inputs and 2 outputs
#     """
#     x = layers.Input(shape=input_shape)

#     # Changes:
#     # Added conv layer before feeding to primary caps layer, with 256 kernels

#     # Layer 1: Conventional Conv2D layer 1
#     conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

#     # Layer 2: Conventional Conv2D layer 2
#     conv2 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv2')(conv1)

#     # Layer 3: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_vector]
#     primarycaps = PrimaryCap(conv2, dim_vector=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

#     # Layer 4: Capsule layer. Dynamic Routing algorithm works here.
#     digitcaps = CapsuleLayer(num_capsule=n_class, dim_vector=16, num_routing=num_routing, name='digit_caps')(primarycaps)

#     # Layer 5: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
#     out_caps = Length(name='out_caps')(digitcaps)

#     # Decoder network.
#     y = layers.Input(shape=(n_class,))
#     masked = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer.
#     x_recon = layers.Dense(512, activation='relu')(masked)
#     x_recon = layers.Dense(1024, activation='relu')(x_recon)
#     x_recon = layers.Dense(784, activation='sigmoid')(x_recon)
#     x_recon = layers.Reshape(target_shape=[28, 28, 1], name='out_recon')(x_recon)

#     # two-input-two-output keras Model
#     return models.Model([x, y], [out_caps, x_recon])


# def margin_loss(y_true, y_pred):
#     """
#     Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
#     :param y_true: [None, n_classes]
#     :param y_pred: [None, num_capsule]
#     :return: a scalar loss value.
#     """
#     L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
#         0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

#     return K.mean(K.sum(L, 1))


# num_classes = 3
# routings = 3

# # define model
# model = CapsNet(input_shape=[28, 28, 1],
#                 n_class=num_classes,
#                 num_routing=routings)
# model.summary()


In [4]:
class Length(layers.Layer):
    """
    Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss.
    Using this layer as model's output can directly predict labels by using `y_pred = np.argmax(model.predict(x), 1)`
    inputs: shape=[None, num_vectors, dim_vector]
    output: shape=[None, num_vectors]
    """
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

    def get_config(self):
        config = super(Length, self).get_config()
        return config


class Mask(layers.Layer):
    """
    Mask a Tensor with shape=[None, num_capsule, dim_vector] either by the capsule with max length or by an additional 
    input mask. Except the max-length capsule (or specified capsule), all vectors are masked to zeros. Then flatten the
    masked Tensor.
    For example:
        ```
        x = keras.layers.Input(shape=[8, 3, 2])  # batch_size=8, each sample contains 3 capsules with dim_vector=2
        y = keras.layers.Input(shape=[8, 3])  # True labels. 8 samples, 3 classes, one-hot coding.
        out = Mask()(x)  # out.shape=[8, 6]
        # or
        out2 = Mask()([x, y])  # out2.shape=[8,6]. Masked with true labels y. Of course y can also be manipulated.
        ```
    """
    def call(self, inputs, **kwargs):
        if type(inputs) is list:  # true label is provided with shape = [None, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of capsules. Mainly used for prediction
            # compute lengths of capsules
            x = K.sqrt(K.sum(K.square(inputs), -1))
            # generate the mask which is a one-hot code.
            # mask.shape=[None, n_classes]=[None, num_capsule]
            mask = K.one_hot(indices=K.argmax(x, 1), num_classes=x.get_shape().as_list()[1])

        # inputs.shape=[None, num_capsule, dim_capsule]
        # mask.shape=[None, num_capsule]
        # masked.shape=[None, num_capsule * dim_capsule]
        masked = K.batch_flatten(inputs * K.expand_dims(mask, -1))
        return masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][1] * input_shape[0][2]])
        else:  # no true label provided
            return tuple([None, input_shape[1] * input_shape[2]])

    def get_config(self):
        config = super(Mask, self).get_config()
        return config


def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors


class CapsuleLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_capsule] and output shape = \
    [None, num_capsule, dim_capsule]. For Dense Layer, input_dim_capsule = dim_capsule = 1.
    
    :param num_capsule: number of capsules in this layer
    :param dim_capsule: dimension of the output vectors of the capsules in this layer
    :param routings: number of iterations for the routing algorithm
    """
    def __init__(self, num_capsule, dim_capsule, routings=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
#         self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)

#         self.kernel_initializer = initializers.random_uniform(-1, 1)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True
    def call(self, inputs, training=None):
        # Expand the input in axis=1, tile in that axis to num_capsule, and 
        #  expands another axis at the end to prepare the multiplication with W.
        #  inputs.shape=[None, input_num_capsule, input_dim_capsule]
        #  inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
        #  inputs_tiled.shape=[None, num_capsule, input_num_capsule, 
        #                            input_dim_capsule, 1]
        inputs_expand = tf.expand_dims(inputs, 1)
        inputs_tiled  = tf.tile(inputs_expand, [1, self.num_capsule, 1, 1])
        inputs_tiled  = tf.expand_dims(inputs_tiled, 4)

        # Compute `W * inputs` by scanning inputs_tiled on dimension 0 (map_fn).
        # - Use matmul (without transposing any element). Note the order!
        # Thus:
        #  x.shape=[num_capsule, input_num_capsule, input_dim_capsule, 1]
        #  W.shape=[num_capsule, input_num_capsule, dim_capsule,input_dim_capsule]
        # Regard the first two dimensions as `batch` dimension,
        # then matmul: [dim_capsule, input_dim_capsule] x [input_dim_capsule, 1]-> 
        #              [dim_capsule, 1].
        #  inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule, 1]

        inputs_hat = tf.map_fn(lambda x: tf.matmul(self.W, x), elems=inputs_tiled)     

        # Begin: Routing algorithm ----------------------------------------------#
        # The prior for coupling coefficient, initialized as zeros.
        #  b.shape = [None, self.num_capsule, self.input_num_capsule, 1, 1].
        b = tf.zeros(shape=[tf.shape(inputs_hat)[0], self.num_capsule,  self.input_num_capsule, 1, 1])

        assert self.routings > 0, 'The routings should be > 0.'
        for i in range(self.routings):
                # Apply softmax to the axis with `num_capsule`
                #  c.shape=[batch_size, num_capsule, input_num_capsule, 1, 1]
                c = layers.Softmax(axis=1)(b)

                # Compute the weighted sum of all the predicted output vectors.
                #  c.shape =  [batch_size, num_capsule, input_num_capsule, 1, 1]
                #  inputs_hat.shape=[None, num_capsule, input_num_capsule,dim_capsule,1]
                # The function `multiply` will broadcast axis=3 in c to dim_capsule.
                #  outputs.shape=[None, num_capsule, input_num_capsule, dim_capsule, 1]
                # Then sum along the input_num_capsule
                #  outputs.shape=[None, num_capsule, 1, dim_capsule, 1]
                # Then apply squash along the dim_capsule
                outputs = tf.multiply(c, inputs_hat)
                outputs = tf.reduce_sum(outputs, axis=2, keepdims=True)
                outputs = squash(outputs, axis=-2)  # [None, 10, 1, 16, 1]

                if i < self.routings - 1:
                      # Update the prior b.
                      #  outputs.shape =  [None, num_capsule, 1, dim_capsule, 1]
                      #  inputs_hat.shape=[None,num_capsule,input_num_capsule,dim_capsule,1]
                      # Multiply the outputs with the weighted_inputs (inputs_hat) and add  
                      # it to the prior b.  
                        outputs_tiled = tf.tile(outputs, [1, 1, self.input_num_capsule, 1, 1])
                        agreement = tf.matmul(inputs_hat, outputs_tiled, transpose_a=True)
                        b = tf.add(b, agreement)

        # End: Routing algorithm ------------------------------------------------#
        # Squeeze the outputs to remove useless axis:
        #  From  --> outputs.shape=[None, num_capsule, 1, dim_capsule, 1]
        #  To    --> outputs.shape=[None, num_capsule,    dim_capsule]
        outputs = tf.squeeze(outputs, [2, 4])
        return outputs
#     def call(self, inputs, training=None):
#         # inputs.shape=[None, input_num_capsule, input_dim_capsule]
#         # inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
#         inputs_expand = K.expand_dims(inputs, 1)

#         # Replicate num_capsule dimension to prepare being multiplied by W
#         # inputs_tiled.shape=[None, num_capsule, input_num_capsule, input_dim_capsule]
#         inputs_tiled = K.tile(inputs_expand, [1, self.num_capsule, 1, 1])

#         # Compute `inputs * W` by scanning inputs_tiled on dimension 0.
#         # x.shape=[num_capsule, input_num_capsule, input_dim_capsule]
#         # W.shape=[num_capsule, input_num_capsule, dim_capsule, input_dim_capsule]
#         # Regard the first two dimensions as `batch` dimension,
#         # then matmul: [input_dim_capsule] x [dim_capsule, input_dim_capsule]^T -> [dim_capsule].
#         # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
#         inputs_hat = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs_tiled)

#         # Begin: Routing algorithm ---------------------------------------------------------------------#
#         # The prior for coupling coefficient, initialized as zeros.
#         # b.shape = [None, self.num_capsule, self.input_num_capsule].
#         b = tf.zeros(shape=[K.shape(inputs_hat)[0], self.num_capsule, self.input_num_capsule])

#         assert self.routings > 0, 'The routings should be > 0.'
#         for i in range(self.routings):
#             # c.shape=[batch_size, num_capsule, input_num_capsule]
#             c = tf.nn.softmax(b, axis=1)

#             # c.shape =  [batch_size, num_capsule, input_num_capsule]
#             # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
#             # The first two dimensions as `batch` dimension,
#             # then matmal: [input_num_capsule] x [input_num_capsule, dim_capsule] -> [dim_capsule].
#             # outputs.shape=[None, num_capsule, dim_capsule]
#             outputs = squash(K.batch_dot(c, inputs_hat, [2, 2]))  # [None, 10, 16]

#             if i < self.routings - 1:
#                 # outputs.shape =  [None, num_capsule, dim_capsule]
#                 # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
#                 # The first two dimensions as `batch` dimension,
#                 # then matmal: [dim_capsule] x [input_num_capsule, dim_capsule]^T -> [input_num_capsule].
#                 # b.shape=[batch_size, num_capsule, input_num_capsule]
#                 b += K.batch_dot(outputs, inputs_hat, [2, 3])
#         # End: Routing algorithm -----------------------------------------------------------------------#

#         return outputs

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

    def get_config(self):
        config = {
            'num_capsule': self.num_capsule,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        }
        base_config = super(CapsuleLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    """
    Apply Conv2D `n_channels` times and concatenate all capsules
    :param inputs: 4D tensor, shape=[None, width, height, channels]
    :param dim_capsule: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_capsule]
    """
    output = layers.Conv2D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides, padding=padding,
                           name='primarycap_conv2d')(inputs)
    
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)


K.set_image_data_format('channels_last')


def CapsNet(input_shape, classes, routings=3):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)
#     conv1 = layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='valid', activation='relu', name='conv1')(x)
#     conv1 = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='valid', activation='relu', name='conv2')(conv1)
#     conv1 = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='valid', activation='relu', name='conv3')(conv1)
    
    
    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    conv1 = layers.BatchNormalization()(conv1)
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')
    primarycaps = layers.BatchNormalization()(primarycaps)
    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=classes, dim_capsule=16, routings=routings,
                             name='digitcaps')(primarycaps)
    digitcaps = layers.BatchNormalization()(digitcaps)
    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='capsnet')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(classes,))
    masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
    masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction

    # Shared Decoder model in training and prediction
    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(512, activation='relu', input_dim=16*classes))
    decoder.add(layers.BatchNormalization())
    
    decoder.add(layers.Dense(1024, activation='relu'))
    decoder.add(layers.BatchNormalization())
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Models for training and evaluation (prediction)
    train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
    eval_model = models.Model(x, [out_caps, decoder(masked)])

    # manipulate model
    noise = layers.Input(shape=(classes, 16))
    noised_digitcaps = layers.Add()([digitcaps, noise])
    masked_noised_y = Mask()([noised_digitcaps, y])
    manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))
    return train_model, eval_model, manipulate_model


def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))


# def train(model, data, args):
#     """
#     Training a CapsuleNet
#     :param model: the CapsuleNet model
#     :param data: a tuple containing training and testing data, like `((x_train, y_train), (x_test, y_test))`
#     :param args: arguments
#     :return: The trained model
#     """
#     # unpacking the data
#     (x_train, y_train), (x_test, y_test) = data

#     # callbacks
# #     log = callbacks.CSVLogger(args.save_dir + '/log.csv')
# #     tb = callbacks.TensorBoard(log_dir=args.save_dir + '/tensorboard-logs',
# #                                batch_size=args.batch_size, histogram_freq=int(args.debug))
#     checkpoint = callbacks.ModelCheckpoint(args.save_dir + '/weights-{epoch:02d}.h5', monitor='val_capsnet_acc',
#                                            save_best_only=True, save_weights_only=True, verbose=1)
#     lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: args.lr * (args.lr_decay ** epoch))

#     # compile the model
#     model.compile(optimizer=optimizers.Adam(lr=args.lr),
#                   loss=[margin_loss, 'mse'],
#                   loss_weights=[1., args.lam_recon],
#                   metrics={'capsnet': 'accuracy'})

#     """
#     # Training without data augmentation:
#     model.fit([x_train, y_train], [y_train, x_train], batch_size=args.batch_size, epochs=args.epochs,
#               validation_data=[[x_test, y_test], [y_test, x_test]], callbacks=[log, tb, checkpoint, lr_decay])
#     """

#     # Begin: Training with data augmentation ---------------------------------------------------------------------#
#     def train_generator(x, y, batch_size, shift_fraction=0.):
#         train_datagen = ImageDataGenerator(width_shift_range=shift_fraction,
#                                            height_shift_range=shift_fraction)  # shift up to 2 pixel for MNIST
#         generator = train_datagen.flow(x, y, batch_size=batch_size)
#         while 1:
#             x_batch, y_batch = generator.next()
#             yield ([x_batch, y_batch], [y_batch, x_batch])

#     # Training with data augmentation. If shift_fraction=0., also no augmentation.
#     model.fit_generator(generator=train_generator(x_train, y_train, args.batch_size, args.shift_fraction),
#                         steps_per_epoch=int(y_train.shape[0] / args.batch_size),
#                         epochs=args.epochs,
#                         validation_data=[[x_test, y_test], [y_test, x_test]],
#                         callbacks=[checkpoint, lr_decay])
#     # End: Training with data augmentation -----------------------------------------------------------------------#

#     model.save_weights(args.save_dir + '/trained_model.h5')
#     print('Trained model saved to \'%s/trained_model.h5\'' % args.save_dir)

#     from utils import plot_log
#     plot_log(args.save_dir + '/log.csv', show=True)

#     return model


# def test(model, data, args):
#     x_test, y_test = data
#     y_pred, x_recon = model.predict(x_test, batch_size=100)
#     print('-'*30 + 'Begin: test' + '-'*30)
#     print('Test acc:', np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1))/y_test.shape[0])

#     img = combine_images(np.concatenate([x_test[:50],x_recon[:50]]))
#     image = img * 255
#     Image.fromarray(image.astype(np.uint8)).save(args.save_dir + "/real_and_recon.png")
#     print()
#     print('Reconstructed images are saved to %s/real_and_recon.png' % args.save_dir)
#     print('-' * 30 + 'End: test' + '-' * 30)
#     plt.imshow(plt.imread(args.save_dir + "/real_and_recon.png"))
#     plt.show()


# def manipulate_latent(model, data, args):
#     print('-'*30 + 'Begin: manipulate' + '-'*30)
#     x_test, y_test = data
#     index = np.argmax(y_test, 1) == args.digit
#     number = np.random.randint(low=0, high=sum(index) - 1)
#     x, y = x_test[index][number], y_test[index][number]
#     x, y = np.expand_dims(x, 0), np.expand_dims(y, 0)
#     noise = np.zeros([1, 10, 16])
#     x_recons = []
#     for dim in range(16):
#         for r in [-0.25, -0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2, 0.25]:
#             tmp = np.copy(noise)
#             tmp[:,:,dim] = r
#             x_recon = model.predict([x, y, tmp])
#             x_recons.append(x_recon)

#     x_recons = np.concatenate(x_recons)

#     img = combine_images(x_recons, height=16)
#     image = img*255
#     Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/manipulate-%d.png' % args.digit)
#     print('manipulated result saved to %s/manipulate-%d.png' % (args.save_dir, args.digit))
#     print('-' * 30 + 'End: manipulate' + '-' * 30)

In [5]:
class CapsuleDataGenerator(keras.utils.Sequence):
        'Generates data for Keras'
        def __init__(self, list_IDs, labels, batch_size=32, dim=(256, 256), n_channels=1,
                     n_classes=10, shuffle=True, resize_setting=(256, 256)):
                'Initialization'
                self.dim = dim
                self.batch_size = batch_size
                self.labels = labels
                self.list_IDs = list(list_IDs)
                self.n_channels = n_channels
                self.n_classes = n_classes
                self.shuffle = shuffle
                self.on_epoch_end()
                self.resize_setting = resize_setting
                

        def __len__(self):
                'Denotes the number of batches per epoch'
                return int(np.ceil(len(self.list_IDs) / self.batch_size))

        def __getitem__(self, index):
                'Generate one batch of data'
                # Generate indexes of the batch
                end = (index+1)*self.batch_size if (index+1)*self.batch_size < len(self.indexes) else len(self.indexes)
                indexes = self.indexes[index*self.batch_size:end] 

                # Find list of IDs
                list_IDs_temp = [self.list_IDs[k] for k in indexes]         
                # Generate data
                X, y = self.__data_generation(list_IDs_temp)
               
                return X, y

        def on_epoch_end(self):
                'Updates indexes after each epoch'
                self.indexes = np.arange(len(self.list_IDs))
                if self.shuffle == True:
                        np.random.shuffle(self.indexes)
        
        def noisy_image(self, noise_typ, image):
                if noise_typ == "gauss":
                          row,col= image.shape
                          mean = 0
                          var = 0.1
                          sigma = var**0.5
                          gauss = np.random.normal(mean,sigma,(row,col))
                          gauss = gauss.reshape(row,col)
                          noisy = image + gauss
                          return noisy

                elif noise_typ == "s&p":
                          row,col = image.shape
                          s_vs_p = 0.5
                          amount = 0.004
                          out = np.copy(image)
                          # Salt mode
                          num_salt = np.ceil(amount * image.size * s_vs_p)
                          coords = [np.random.randint(0, i - 1, int(num_salt))
                                  for i in image.shape]
                          out[coords] = 1

                          # Pepper mode
                          num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
                          coords = [np.random.randint(0, i - 1, int(num_pepper))
                                  for i in image.shape]
                          out[coords] = 0
                          return out

                elif noise_typ == "poisson":
                          vals = len(np.unique(image))
                          vals = 2 ** np.ceil(np.log2(vals))
                          noisy = np.random.poisson(image * vals) / float(vals)
                          return noisy

                elif noise_typ == "speckle":
                          row,col = image.shape
                          gauss = np.random.randn(row,col)
                          gauss = gauss.reshape(row,col)        
                          noisy = image + image * gauss
                          return noisy
            
        def __data_generation(self, list_IDs_temp):
            
                # Initialization
                image_array_size = ( self.batch_size, *self.dim, self.n_channels )
                X = np.zeros(image_array_size, dtype=np.uint8)
                y = np.array([0]* self.batch_size)
                
                for i, ID in enumerate(list_IDs_temp):
                        image = cv2.imread(ID, 0)
                        image = image.astype('float32') / 255
                        image = cv2.resize(image, (self.resize_setting[1], self.resize_setting[0]))
                        
#                         from random import sample
#                         random_index = sample(range(0, 32), k=8)
                
#                         if i in random_index: image = self.noisy_image("gauss", image)
                        image = np.reshape(image, (*image.shape, 1))
                        X[i,], y[i] =  image, self.labels[ID]
                        
                        x_batch, y_batch = X, to_categorical(y, num_classes=self.n_classes)
                return ([x_batch, y_batch], [y_batch, x_batch])
            
def make_cap_generator(dataset, batch_size=32):
        common_params = {
                              'batch_size': batch_size,
                              'n_classes' : len(np.unique(dataset.Class)),
                              'n_channels': 1,
                              'shuffle'   : False,
                              'resize_setting': (100, 90),
                              'dim': (100, 90)
        }
        
        dataset_dict = collections.OrderedDict(zip(dataset.Path, dataset.Class))
        dataset_generator = CapsuleDataGenerator(dataset["Path"], dataset_dict ,**common_params)
        return dataset_generator

In [6]:
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K
from datetime import datetime

# from Dental_Tool.Inception_v3 import Custom_Inception_V3, Simple_Inception
# from Dental_Tool.Inception import create_inception_v4
from Dental_Tool.Data_processing import *
# from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v2 import *

import keras.backend as K 
import matplotlib.pyplot as plt
import numpy as np
import keras
import time
# from Dental_Tool.Capsule import CapsNet

root_dir  =  "Capsnet_Fold_5__Testtttt___________" + datetime.now().strftime("%Y%m%d-%H%M%S")
init_directory(f"Results/{root_dir}/table")
model_info = ("Capsnet", CapsNet)
performance_list = []
epochs     = 5
Fold_num = 5
num_params = 1
fold_data_num = 1

        
        
for train_dataset, valid_dataset, test_dataset, train_generator, valid_generator, test_generator in\
        K_Fold_balance_data_generator(dataset, argscale=argscale_num, k_fold_num=Fold_num):
        
        cap_train_gen = make_cap_generator(train_dataset, 64)
        cap_valid_gen = make_cap_generator(valid_dataset, 64)
        cap_test_gen  = make_cap_generator(test_dataset , 64)
        
        
        K.clear_session()
        print(f"Training dataset: {len(train_dataset)},Validation dataset: {len(valid_dataset)}, Testing dataset: {len(test_dataset)}" )
        
        init_directory(f"Results/{root_dir}/Fold_{fold_data_num}/table")
           
        train_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/train_dataset.csv", index=False)
        valid_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/valid_dataset.csv", index=False)
        test_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/test_dataset.csv"  , index=False)
        
        print(set(train_dataset.source) & set(valid_dataset.source))
        print(set(valid_dataset.source) & set(test_dataset.source))
        print(set(train_dataset.source) & set(test_dataset.source))
        
        K.clear_session()
        best_models = None
        
        model_name, model_fnc = model_info
        
        K.clear_session()

        param_dir  = f"{model_name}_Fold_{fold_data_num}"
        print(f"Fold {fold_data_num}: loading {model_name} ......")

        start = time.time()
        model, eval_model, manipulate_model  = model_fnc(input_shape=(100, 90, 1), classes=len(np.unique(train_dataset["Class"])))
        
        model.compile(optimizer=optimizers.Adam(lr=0.01),
                      loss=[margin_loss, 'mse'],
                      loss_weights=[ 1, 0.005],
                      metrics={'capsnet': 'accuracy'})
        model.summary()
        end = time.time()
        elapse = end - start
        print(f"Fold {fold_data_num}: loading Done, cost {elapse} seconds")


        init_directory(f"Results/{root_dir}/Fold_{fold_data_num}/{param_dir}/parameter")

        filepath="Results/%s/Fold_%d/%s/parameter/weights-improvement-{epoch:02d}-{val_accuracy:.4f}.hdf5" \
                    %(root_dir, fold_data_num, param_dir)
        
        from keras import callbacks
        checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: 0.001 * np.exp(-epoch / 10.))

        # change 
        history = model.fit_generator(
                                generator=cap_train_gen,
                                validation_data=cap_valid_gen,
                                epochs=epochs,
                                workers=2,
                                shuffle=True,
                                verbose=1,
                                callbacks=[ checkpoint, lr_decay ],
                               )
        
        plot_result(history, f"Results/{root_dir}/Fold_{fold_data_num}/{param_dir}")

        K.clear_session()
        top_k_param = get_k_top_value(f"Results/{root_dir}/Fold_{fold_data_num}/{param_dir}/parameter",  k_th = num_params )
        best_param = top_k_param[0]
        
        K.clear_session()
        print(f"loading {best_param} ......")
        start = time.time()
        best_model = load_model(best_param)
        end = time.time()
        elapse = end - start
        print(f"loading Done, cost {elapse} seconds")

        test_predict, rec_image = best_model.predict_generator(cap_test_gen, verbose=1)
        test_predict = test_predict[:len(test_dataset)]
        
        test_result  = np.argmax(test_predict, axis=1)
        test_dataset["Predict"] = test_result
        
        K.clear_session()
        
        confusion = confusion_matrix(test_dataset.Class.astype('int'), test_result.astype('int'))
        curr_acc = (test_dataset.Class == test_dataset.Predict).mean() * 100
        title = "Accuracy  = {:5.2f} %".format(curr_acc)
        print(title)
        init_directory(f"Results/{root_dir}/Fold_{fold_data_num}/table")
        plot_confusion_matrix(confusion, len(np.unique(test_dataset.Class)), curr_acc, f"Results/{root_dir}/Fold_{fold_data_num}", 0)
        
        test_predict = np.round(test_predict, 2)
        test_dataset[f"{model_name}_Prob"] = pd.Series(test_predict.tolist())
        test_dataset["Predict"] = test_result
        
        test_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/test_predict.csv", index=False)
        print(statistic(test_dataset))
        K.clear_session()
        
        performance_list.append(curr_acc)
        fold_data_num += 1

stage 0: 58560, stage 1: 105040, stage 2: 46080, stage 3: 19120
stage 0: 19520, stage 1: 34960, stage 2: 15040, stage 3: 6240
stage 0: 19520, stage 1: 35200, stage 2: 15440, stage 3: 6720
Training dataset: 57360,Validation dataset: 18720, Testing dataset: 20160
set()
set()
set()
Fold 1: loading Capsnet ......


ValueError: Inconsistent shapes: saw (40128, 3, 1, 3, 16) but expected (40128, 3, 1, 16) 